# Convolutional Neural Networks for Dogs and Cats

## Preparation (40 points total)
#### [10 points] Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis. Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).  

In [34]:
%%time
from PIL import Image
import pandas as pd
import numpy as np

image_size = (20, 20)
X = []
y = []
label = [0, 1] #cat == 0, dog == 1
for i in range(0, 8000):
    image = Image.open('train/cat.' + str(i) +'.jpg')
    image = image.resize(image_size)
    image = image.convert('L')
    image_array = np.array(image).reshape(400,)
    X.append(image_array)
    y.append(0)
    
for i in range(0, 8000):
    image = Image.open('train/dog.' + str(i) +'.jpg')
    image = image.resize(image_size)
    image = image.convert('L')
    image_array = np.array(image).reshape(400,)
    X.append(image_array)
    y.append(1)

X = np.asarray(X)
y = np.asarray(y)

Wall time: 1min 8s


In [35]:
print(X.shape)
print(y.shape)

X = X.astype(np.float32) / 255.0 # normalize

(16000, 400)
(16000,)


#### [15 points] Choose and explain what metric(s) you will use to evaluate your algorithm’s performance. You should give a detailed argument for why this (these) metric(s) are appropriate on your data. That is, why is the metric appropriate for the task (e.g., in terms of the business case for the task). Please note: rarely is accuracy the best evaluation metric to use. Think deeply about an appropriate measure of performance.

#### [15 points] Choose the method you will use for dividing your data into training and testing (i.e., are you using Stratified 10-fold cross validation? Shuffle splits? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. Convince me that your cross validation method is a realistic mirroring of how an algorithm would be used in practice.

In [47]:
import keras
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

y_ohe_train = keras.utils.to_categorical(y_train, 2)


## Modeling (50 points total)
#### [20 points] Create a convolutional neural network to use on your data using tensorflow. 

In [52]:
%%time
cnn2 = Sequential()
cnn2.add(Reshape((1, 20, 20), input_shape=(1, 400)))
cnn2.add(Conv2D(filters=16, kernel_size=(2, 2), padding='same'))
cnn2.add(Activation('relu'))
cnn2.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))


cnn2.add(Flatten())
cnn2.add(Dense(2))
cnn2.add(Activation('softmax'))

cnn2.compile(loss='mean_squared_logarithmic_error',
            optimizer='adam',
            metrics=['accuracy'])

cnn2.fit(np.expand_dims(X_train, axis=1), y_ohe_train, batch_size=32, epochs=50, shuffle=True, verbose=0)

Wall time: 50.8 s


In [53]:
from sklearn import metrics as mt

yhat2 = np.argmax(cnn2.predict(np.expand_dims(X_test, axis=1)), axis=1)
acc2 = mt.accuracy_score(y_test, yhat)
acc2

0.65749999999999997

#### [20 points] Investigate at least two different deep network architectures. Use the method of cross validation and evaluation metric that you argued for at the beginning of the lab. Visualize the results of the CNNs. 

In [64]:
%%time
cnn = Sequential()

cnn.add(Reshape((1, 20, 20), input_shape=(1, 400)))
cnn.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

cnn.add(Dropout(.2))
cnn.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(Dropout(.2))
cnn.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

cnn.add(Dropout(.2))
cnn.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(Dropout(.2))
cnn.add(Conv2D(filters=256, kernel_size=(3, 3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))


cnn.add(Flatten())
cnn.add(Dense(2))
cnn.add(Activation('softmax'))

cnn.compile(loss='mean_squared_logarithmic_error',
            optimizer='adam',
            metrics=['accuracy'])

cnn.fit(np.expand_dims(X_train, axis=1), y_ohe_train, batch_size=32, epochs=75, shuffle=True, verbose=1)

Epoch 1/75
12800/12800 [==============================] - 8s - loss: 0.1229 - acc: 0.5307     
Epoch 2/75
12800/12800 [==============================] - 7s - loss: 0.1194 - acc: 0.5809     
Epoch 3/75
12800/12800 [==============================] - 7s - loss: 0.1170 - acc: 0.5973     
Epoch 4/75
12800/12800 [==============================] - 7s - loss: 0.1159 - acc: 0.6019     
Epoch 5/75
12800/12800 [==============================] - 7s - loss: 0.1148 - acc: 0.6116     
Epoch 6/75
12800/12800 [==============================] - 7s - loss: 0.1141 - acc: 0.6138     
Epoch 7/75
12800/12800 [==============================] - 7s - loss: 0.1134 - acc: 0.6205     
Epoch 8/75
12800/12800 [==============================] - 7s - loss: 0.1129 - acc: 0.6253     
Epoch 9/75
12800/12800 [==============================] - 7s - loss: 0.1115 - acc: 0.6308     
Epoch 10/75
12800/12800 [==============================] - 7s - loss: 0.1115 - acc: 0.6313     
Epoch 11/75
12800/12800 [========================

In [65]:
yhat = np.argmax(cnn.predict(np.expand_dims(X_test, axis=1)), axis=1)
acc = mt.accuracy_score(y_test, yhat)
acc

0.65874999999999995

#### [10 points] Compare the performance of your convolutional network to a standard multi-layer perceptron (MLP) using the receiver operating characteristic and area under the curve.

## Exceptional Work (10 points total)
#### One idea: Visualize the convolutional filters chosen by your CNN. Try to interpret some of their meanings.